In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset,
import numpy as np
from tensorflow.keras.datasets import imdb
from sklearn.model_selection import train_test_split

In [ ]:
(train_data, train_label), (test_data,test_label) = imdb.load_data(num_words=10000)

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
word_index = imdb.get_word_index()
reserve_word_index = dict([(value, key) for (key, value) in word_index.items()])
decoded_review = ' '.join(reserve_word_index.get(i-3, '?') for i in train_data[0])

len(word_index)

1641221/1641221 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


88584

In [ ]:
def vectorize_sequences(sequences, dim=10000):
  vectors = np.zeros((len(sequences), dim), dtype=np.float32)
  for i, sequence in enumerate(sequences):
    vectors[i, sequence] = 1.0
  return vectors


In [ ]:
x_train = vectorize_sequences(train_data)
y_train = np.asarray(train_label).astype('float32')
x_test = vectorize_sequences(test_data)
y_test = np.asarray(test_label).astype('float32')

In [ ]:
x_train = torch.tensor(x_train)
y_train = torch.tensor(y_train).unsqueeze(1)
x_test = torch.tensor(x_test)
y_test = torch.tensor(y_test).unsqueeze(1)

In [ ]:
x_val = x_train[:10000]
y_val = y_train[:10000]
x_train = x_train[10000:]
y_train = y_train[10000:]

In [ ]:
train_dataset =  TensorDataset(x_train, y_train)
val_dataset = TensorDataset(x_val, y_val)
train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=512)

In [ ]:
y_train.shape

torch.Size([15000, 1])

In [ ]:
class BinaryClassificationModel(nn.Module):
  def __init__(self):
    super(BinaryClassificationModel, self).__init__()
    self.fc1 = nn.Linear(10000, 16)
    self.relu = nn.ReLU()
    self.fc2 = nn.Linear(16, 16)
    self.fc3 = nn.Linear(16, 1)
    self.sigmoid = nn.Sigmoid()

  def forward(self, x):
    x = self.fc1(x)
    x = self.relu(x)
    x = self.fc2(x)
    x = self.relu(x)
    x = self.fc3(x)
    x = self.sigmoid(x)
    return x

model = BinaryClassificationModel()
criterion = nn.BCELoss()
optimizer = optim.RMSprop(model.parameters(), lr=0.001)
epochs = 20

for epoch in range(epochs):
  model.train()
  running_loss = 0.0
  correct_train = 0
  total_train = 0

  for inputs, labels in train_loader:
    optimizer.zero_grad()
    outputs = model(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()

    preds = (outputs >= 0.5).float()
    correct_train += (preds == labels).sum().item()
    total_train += labels.size(0)

  train_accuracy = correct_train / total_train

  model.eval()
  val_loss = 0.0
  correct_val = 0
  total_val = 0

  with torch.no_grad():
    for val_inputs, val_labels in val_loader:
      val_outputs = model(val_inputs)
      val_loss += criterion(val_outputs, val_labels).item()

      val_preds = (val_outputs >= 0.5).float()
      correct_val += (val_preds == val_labels).sum().item()
      total_val += val_labels.size(0)

  val_accuracy = correct_val / total_val
  print(f"Epoch {epoch+1}/{epochs} - Loss: {running_loss:.4f} - Acc: {train_accuracy:.4f} - Val Loss: {val_loss:.4f} - Val Acc: {val_accuracy:.4f}")


Epoch 1/20 - Loss: 14.7354 - Acc: 0.7891 - Val Loss: 6.9505 - Val Acc: 0.8801
Epoch 2/20 - Loss: 7.7982 - Acc: 0.9159 - Val Loss: 5.6507 - Val Acc: 0.8917
Epoch 3/20 - Loss: 5.5606 - Acc: 0.9394 - Val Loss: 5.4555 - Val Acc: 0.8901
Epoch 4/20 - Loss: 4.3614 - Acc: 0.9547 - Val Loss: 5.5772 - Val Acc: 0.8867
Epoch 5/20 - Loss: 3.5588 - Acc: 0.9645 - Val Loss: 5.7255 - Val Acc: 0.8892
Epoch 6/20 - Loss: 2.8751 - Acc: 0.9739 - Val Loss: 7.0223 - Val Acc: 0.8719
Epoch 7/20 - Loss: 2.3807 - Acc: 0.9800 - Val Loss: 6.3866 - Val Acc: 0.8833
Epoch 8/20 - Loss: 1.9453 - Acc: 0.9848 - Val Loss: 6.8828 - Val Acc: 0.8811
Epoch 9/20 - Loss: 1.5212 - Acc: 0.9895 - Val Loss: 7.7630 - Val Acc: 0.8781
Epoch 10/20 - Loss: 1.3105 - Acc: 0.9911 - Val Loss: 8.0775 - Val Acc: 0.8754
Epoch 11/20 - Loss: 0.9671 - Acc: 0.9947 - Val Loss: 8.5154 - Val Acc: 0.8765
Epoch 12/20 - Loss: 0.7633 - Acc: 0.9968 - Val Loss: 9.7912 - Val Acc: 0.8690
Epoch 13/20 - Loss: 0.6054 - Acc: 0.9983 - Val Loss: 9.9145 - Val Acc: 0